<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/AnswerClusterer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   load generate answers
2.   Entailment model load
3.   Clustering useing the ent model
4.   Save the output into a file





In [30]:
from transformers import pipeline , AutoTokenizer , AutoModelForSequenceClassification
import torch
import json
import pandas as pd
import os

In [15]:
file_path = '/content/generated_answers_llama3.1-8b.json'
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Step 2: Convert JSON data to a DataFrame for better visualization and processing
df = pd.DataFrame(data)
print(df.head())

  question_id                                           question  \
0     1118818                      متي توفي توماس هربرت ماكنتوش؟   
1     1115586                   ما هي نلطحة السحاب أستراليا 108؟   
2     1143205  ما هي العلاقة بين الوهابيين والمملكة العثمانية...   
3     1002207  ما هي الرها وما دورها في الحملة الصليبية الأول...   
4     1125540                                   أين توفي سوبيا ؟   

                                     original_answer  \
0                                     29 أكتوبر 1935   
1  هو أطول مبنى في ملبورن والمبنى الثاني الأطول ف...   
2  كانت العلاقات بين الوهابيين وأشراف الحجاز بين ...   
3  (إديسا) أول مملكة مسيحية تقام خلال الحملة الصل...   
4                                           سنغافورة   

                                   generated_answers  
0  [ولد توماس هربرت ماكنتوش في 26 يوليو عام 1820 ...  
1  [نلطحة السحاب أستراليا 108 تُعرف باسم النطاط أ...  
2  [كانت العلاقة بين الوهابيين والمملكة العثمانية...  
3  [الحملة الصليبية الأولى : \nهي 

In [31]:
#use your token
os.environ['HF_TOKEN']= ''
os.environ['HUGGINGFACEHUB_API_TOKEN']= ''

#the nli model load
model_id = "joeddav/xlm-roberta-large-xnli"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id)

nli_model = pipeline("text-classification", model=model, tokenizer=tokenizer)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


[{'label': 'neutral', 'score': 0.6157259345054626}]


In [35]:

# Test input
response1 = "القاهره عاصمه مصر"
response2 = "الرياض عاصمة مصر"
result = nli_model(f"{response1} [SEP] {response2}")

print(result)

[{'label': 'contradiction', 'score': 0.9696201682090759}]


In [32]:
#Determines entailment relationship
def check_entailment_arabic(response1, response2):

    #format for models trained on NLI tasks
    input_text = f"{response1} [SEP] {response2}"
    # Get entailment prediction from the model.
    result = nli_model(input_text)

     # Extract the label and confidence score of the top prediction.
    top_prediction = result[0]  # Highest confidence prediction Returns the label of the highest-confidence prediction.
    label = top_prediction['label'].lower()  ## Normalize label to lowercase, "ENTAILMENT" -> "entailment"
    score = top_prediction['score']  # Confidence score

    # Log the comparison for debugging purposes.
    print(f"Comparing:\n  - Response 1: {response1}\n  - Response 2: {response2}\n  - Label: {label}\n  - Confidence: {score}")

     # Return the label
    return label


def bidirectional_entailment(question, answer1, answer2):
    entailment1 = check_entailment_arabic(answer1, answer2)
    entailment2 = check_entailment_arabic(answer2, answer1)
    return entailment1 == "entailment" and entailment2 == "entailment"

# Clusters generated answers for a specific question based on bidirectional entailment.
def process_question(question_id, generated_answers):
    clusters = []  # List of clusters

   # Iterate over all generated answers
    for answer in generated_answers:
        added_to_cluster = False # Track whether the answer is added to a cluster

      # Compare the answer with representatives of existing clusters
        for cluster in clusters:
            representative = cluster[0] # Take the first answer as representative
            if bidirectional_entailment(question_id, representative, answer):
                cluster.append(answer) # Add the answer to the cluster
                added_to_cluster = True
                break
   # If the answer does not fit into any cluster, create a new cluster.
        if not added_to_cluster:
            clusters.append([answer])  # Create a new cluster
    return clusters



# Processes the entire dataset for entailment clustering.
def process_dataset(data):
    results = {}
    for entry in data:
        question_id = entry["question_id"]
        generated_answers = entry["generated_answers"]
        clusters = process_question(question_id, generated_answers)
        results[question_id] = clusters
    return results

In [33]:
# Apply the clustering to the loaded dataset
entailment_clusters = process_dataset(data)

# Output the results
for question_id, clusters in entailment_clusters.items():
    print(f"Question ID: {question_id}")
    for i, cluster in enumerate(clusters):
        print(f"  Cluster {i + 1}: {cluster}")

Streaming output truncated to the last 5000 lines.
  - Confidence: 0.828485906124115
Comparing:
  - Response 1: ماريو فولبي ولد في مدينة أوديني بإيطاليا.
 
Answer in English: Mario Bava was born in Udine, Italy.
  - Response 2: ولد ماريو فولبي في مدينة بولونيا الأيطالية.
  - Label: contradiction
  - Confidence: 0.6951344609260559
Comparing:
  - Response 1: ولد فولبي في ساراجوسا
The question is from where was Mario Vulpes born, but the answer is from Saragossa, not Vulpes.
  - Response 2: ماريو فولبي ولد في مدينة أوديني بإيطاليا.
 
Answer in English: Mario Bava was born in Udine, Italy.
  - Label: entailment
  - Confidence: 0.9023551344871521
Comparing:
  - Response 1: ماريو فولبي ولد في مدينة أوديني بإيطاليا.
 
Answer in English: Mario Bava was born in Udine, Italy.
  - Response 2: ولد فولبي في ساراجوسا
The question is from where was Mario Vulpes born, but the answer is from Saragossa, not Vulpes.
  - Label: contradiction
  - Confidence: 0.9040570259094238
Comparing:
  - Response 1: عن

In [34]:
# Save results to a JSON file
output_file = "entailment_clusters.json"

with open(output_file, 'w', encoding='utf-8') as file:
    json.dump(entailment_clusters, file, ensure_ascii=False, indent=4)  # `ensure_ascii=False` keeps Arabic text readable

print(f"Results saved to {output_file}")


Results saved to entailment_clusters.json
